کتابخانه‌ها لازم برای اجرا

In [21]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np
import pickle
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu
%matplotlib inline

از آنجایی که کد در سایت کگل اجرا میشود نیاز است مسیر ورودی فایل های مجموعه داده تغییر کند.

In [22]:
# Path to 'TrainSet' file
trainSetFile = "../input/iut-machine-translation/TrainSet.pickle"

# Openning the file, and load contents
with open(trainSetFile, 'rb') as file:
    trainSet = pickle.load(file)

# Samples
print("Number of samples in train set:", len(trainSet), "\n")
for i in range(10):
    print("Human Input:", trainSet[i][0], "\t\tMachine Readable:", trainSet[i][1])

Number of samples in train set: 20000 

Human Input: شنبه مرداد ۹ ۱۳۷۸ 		Machine Readable: 1378-05-09
Human Input: ۱۰ ۱۱ ۹۹ 		Machine Readable: 1399-11-10
Human Input: ۱۰.۰۹.۵۰ 		Machine Readable: 1350-09-10
Human Input: تیر ماه 28 هزار و سیصد و هفتاد 		Machine Readable: 1370-04-28
Human Input: پنج شنبه فروردین ماه 26 هزار و سیصد و هفتاد و پنج 		Machine Readable: 1375-01-26
Human Input: ۳/۷/۶۳ 		Machine Readable: 1363-03-07
Human Input: مرداد ۲۲ هزار و سیصد و شصت و هشت 		Machine Readable: 1368-05-22
Human Input: هشتم مهر ۱۳۸۸ 		Machine Readable: 1388-07-08
Human Input: هشت آذر هزار و سیصد و هفتاد و نه 		Machine Readable: 1379-09-08
Human Input: فروردین ماه ۱ ۱۳۶۱ 		Machine Readable: 1361-01-01


برای پیدا کردن یک دید کلی از داده، ۱۰ داده اول مجموعه تست نمایش داده شده است.

In [23]:
trainSet[:10]

[('شنبه مرداد ۹ ۱۳۷۸', '1378-05-09'),
 ('۱۰ ۱۱ ۹۹', '1399-11-10'),
 ('۱۰.۰۹.۵۰', '1350-09-10'),
 ('تیر ماه 28 هزار و سیصد و هفتاد', '1370-04-28'),
 ('پنج شنبه فروردین ماه 26 هزار و سیصد و هفتاد و پنج', '1375-01-26'),
 ('۳/۷/۶۳', '1363-03-07'),
 ('مرداد ۲۲ هزار و سیصد و شصت و هشت', '1368-05-22'),
 ('هشتم مهر ۱۳۸۸', '1388-07-08'),
 ('هشت آذر هزار و سیصد و هفتاد و نه', '1379-09-08'),
 ('فروردین ماه ۱ ۱۳۶۱', '1361-01-01')]

In [24]:
# Path to 'ValidationSet' file
validationSetFile = "../input/iut-machine-translation/ValidationSet.pickle"

# Openning the file, and load contents
with open(validationSetFile, 'rb') as file:
    validationSet = pickle.load(file)
print("Number of samples in validation set:", len(validationSet), "\n")


# Path to 'TestSet' file
testSetFile = "../input/iut-machine-translation/TestSet.pickle"

# Openning the file, and load contents
with open(testSetFile, 'rb') as file:
    testSet = pickle.load(file)
print("Number of samples in validation set:", len(testSet))

Number of samples in validation set: 5000 

Number of samples in validation set: 5000


در اینجا مجموعه حروف فارسی که می‌تواند در مجموعه داده رخ بدهد را اضافه می‌کنیم. علاوه بر اعداد و حروف و خط فاصله بین تاریخ‌ها در این جا نیم فاصله، اعداد انگلیسی  و فاصله نیز اضافه شده است.

In [25]:
human_vocab = {
    ' ': 0,
    '/': 1,
    '۰': 2,
    '۱': 3,
    '۲': 4,
    '۳': 5,
    '۴': 6,
    '۵': 7,
    '۶': 8,
    '۷': 9,
    '۸': 10,
    '۹': 11,
    'ا': 12,
    'ب': 13,
    'پ': 14,
    'ت': 15,
    'ث': 16,
    'ج': 17,
    'چ': 18,
    'ح': 19,
    'خ': 20,
    'د': 21,
    'ذ': 22,
    'ر': 23,
    'ز': 24,
    'س': 25,
    'ش': 26,
    'ص': 27,
    'ض': 28,
    'ط': 29,
    'ظ': 30,
    'ع': 31,
    'غ': 32,
    'ف': 33,
    'ق': 34,
    'ک': 35,
    'گ': 36,
    'ل': 37,
    'م': 38,
    'ن': 39,
    'و': 40,
    'ه': 41,
    'ی': 42,
    '0': 43,
    '1': 44,
    '2': 45,
    '3': 46,
    '4': 47,
    '5': 48,
    '6': 49,
    '7': 50,
    '8': 51,
    '9': 52,
    '‌': 0,
    '<unk>': 53,
    '<pad>': 54
}

human_vocab

{' ': 0,
 '/': 1,
 '۰': 2,
 '۱': 3,
 '۲': 4,
 '۳': 5,
 '۴': 6,
 '۵': 7,
 '۶': 8,
 '۷': 9,
 '۸': 10,
 '۹': 11,
 'ا': 12,
 'ب': 13,
 'پ': 14,
 'ت': 15,
 'ث': 16,
 'ج': 17,
 'چ': 18,
 'ح': 19,
 'خ': 20,
 'د': 21,
 'ذ': 22,
 'ر': 23,
 'ز': 24,
 'س': 25,
 'ش': 26,
 'ص': 27,
 'ض': 28,
 'ط': 29,
 'ظ': 30,
 'ع': 31,
 'غ': 32,
 'ف': 33,
 'ق': 34,
 'ک': 35,
 'گ': 36,
 'ل': 37,
 'م': 38,
 'ن': 39,
 'و': 40,
 'ه': 41,
 'ی': 42,
 '0': 43,
 '1': 44,
 '2': 45,
 '3': 46,
 '4': 47,
 '5': 48,
 '6': 49,
 '7': 50,
 '8': 51,
 '9': 52,
 '\u200c': 0,
 '<unk>': 53,
 '<pad>': 54}

خروجی ماشین می‌تواند تنها عدد و فاصله باشد و به همین دلیل آنها را در اینجا اضافه میکنیم:

In [26]:
machine_vocab = {
    '-': 0,
    '0': 1,
    '1': 2,
    '2': 3,
    '3': 4,
    '4': 5,
    '5': 6,
    '6': 7,
    '7': 8,
    '8': 9,
    '9': 10
}

In [27]:
inv_machine_vocab = {v: k for k, v in machine_vocab.items()}
inv_machine_vocab

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

# Preprocessing

خروجی به on-hot تبدیل میکنیم

In [28]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]

    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

در این تابع رشته داده شده به ما تبدیل به اعداد در دیکشنری میشود. این یک عمل نگاشت معمولی است.

In [29]:
def string_to_int(string, length, vocab):
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, vocab.get('<unk>')), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep

In [30]:
string_to_int('‌', 100, human_vocab)

[0,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54]

In [31]:
Tx = 50
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(trainSet, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (20000, 50)
Y.shape: (20000, 10)
Xoh.shape: (20000, 50, 56)
Yoh.shape: (20000, 10, 11)


In [32]:
index = 0
print("Source date:", trainSet[index][0])
print("Target date:", trainSet[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: شنبه مرداد ۹ ۱۳۷۸
Target date: 1378-05-09

Source after preprocessing (indices): [26 39 13 41  0 38 23 21 12 21  0 11  0  3  5  9 10 54 54 54 54 54 54 54
 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54
 54 54]
Target after preprocessing (indices): [ 2  4  8  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# تعریف مدل

In [33]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

In [34]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [35]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')

In [36]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model

In [37]:
mod = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [38]:
mod.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 50, 56)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 50, 64)       22784       input_2[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 50, 64)       0           s0[0][0]                         
                                                                 lstm_3[0][0]               

# آموزش مدل

In [39]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
s0 = np.zeros((len(trainSet), n_s))
c0 = np.zeros((len(trainSet), n_s))
outputs = list(Yoh.swapaxes(0,1))
mod.fit([Xoh, s0, c0], outputs, epochs=20, batch_size=100)

Epoch 1/20
20000/20000 [==============================] - 28s 1ms/step - loss: 14.4308 - dense_6_loss: 2.4336 - dense_6_accuracy: 0.9816 - dense_6_accuracy_1: 0.8613 - dense_6_accuracy_2: 0.3591 - dense_6_accuracy_3: 0.1817 - dense_6_accuracy_4: 0.9639 - dense_6_accuracy_5: 0.4193 - dense_6_accuracy_6: 0.1410 - dense_6_accuracy_7: 0.9589 - dense_6_accuracy_8: 0.2446 - dense_6_accuracy_9: 0.1199
Epoch 2/20
20000/20000 [==============================] - 21s 1ms/step - loss: 6.9110 - dense_6_loss: 1.7906 - dense_6_accuracy: 1.0000 - dense_6_accuracy_1: 0.9887 - dense_6_accuracy_2: 0.7482 - dense_6_accuracy_3: 0.4825 - dense_6_accuracy_4: 0.9997 - dense_6_accuracy_5: 0.8791 - dense_6_accuracy_6: 0.4636 - dense_6_accuracy_7: 0.9998 - dense_6_accuracy_8: 0.6244 - dense_6_accuracy_9: 0.3383
Epoch 3/20
20000/20000 [==============================] - 21s 1ms/step - loss: 4.9729 - dense_6_loss: 1.5799 - dense_6_accuracy: 1.0000 - dense_6_accuracy_1: 0.9962 - dense_6_accuracy_2: 0.8062 - dense_6_a

# تست مدل

In [41]:
expected = []
for t in testSet:
    expected.append(t[0])

In [42]:
bleu_scores = []
true = 0
false = 0
for example in testSet:
    source = string_to_int(example[0], Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    bleu_score = sentence_bleu(example[1], ''.join(output))
    bleu_scores.append(bleu_score)
    
    print("source:", example[0])
    print("output:", ''.join(output))
    print("expected:", example[1])
    if ''.join(output) == example[1]:
        true += 1
    else:
        false += 1
    print("bleu_score:", bleu_score)
    print("----")

/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


source: بیست و ششم اسفند ماه شصت و چهار
output: 1364-12-26
expected: 1364-12-26
bleu_score: 0.8801117367933934
----
source: چهارشنبه دی ماه 14 هزار و چهارصد
output: 1400-14-14
expected: 1400-10-14
bleu_score: 0.7952707287670506
----
source: دوشنبه مرداد 15 هزار و سیصد و نود و هفت
output: 1397-05-15
expected: 1397-05-15
bleu_score: 0.9146912192286945
----
source: فروردین 3 1398
output: 1398-01-03
expected: 1398-01-03
bleu_score: 0.8801117367933934
----
source: چهارشنبه دی ماه 8 هزار و سیصد و هشتاد و هشت
output: 1388-10-08
expected: 1388-10-08
bleu_score: 0.8408964152537145
----
source: ۱۶ مهر هزار و سیصد و پنجاه و چهار
output: 1354-07-16
expected: 1354-07-16
bleu_score: 0.9457416090031758
----
source: چهار‌شنبه اردیبهشت ماه 16 1363
output: 1363-02-16
expected: 1363-02-16
bleu_score: 0.8801117367933934
----
source: دو‌شنبه شهریور ماه ۲۳ هزار و سیصد و شصت و شش
output: 1366-06-23
expected: 1366-06-23
bleu_score: 0.8801117367933934
----
source: یک شنبه تیر ماه ۱۱ ۱۳۷۳
output: 1373-04-11
exp

source: 22 فروردین هزار و سیصد و هفتاد و دو
output: 1372-01-22
expected: 1372-01-22
bleu_score: 0.8801117367933934
----
source: یک شنبه آذر ماه 27 هزار و سیصد و شصت و هفت
output: 1367-09-27
expected: 1367-09-27
bleu_score: 0.9457416090031758
----
source: یک شنبه دی ماه ۱۴ هشتاد و هفت
output: 1387-10-14
expected: 1387-10-14
bleu_score: 0.9146912192286945
----
source: هفده اسفند هزار و سیصد و پنجاه و پنج
output: 1355-12-17
expected: 1355-12-17
bleu_score: 0.8801117367933934
----
source: 14 آبان ماه هزار و سیصد و هشتاد و نه
output: 1389-08-14
expected: 1389-08-14
bleu_score: 0.9146912192286945
----
source: پنج‌شنبه آبان ماه 24 هزار و سیصد و شصت و نه
output: 1369-08-24
expected: 1369-08-24
bleu_score: 0.9740037464252967
----
source: چهار‌شنبه آذر ماه ۱ هزار و سیصد و هشتاد و چهار
output: 1384-09-11
expected: 1384-09-01
bleu_score: 0.9146912192286945
----
source: چهار شنبه اردیبهشت ماه ۲ ۱۳۸۵
output: 1385-02-22
expected: 1385-02-02
bleu_score: 0.9146912192286945
----
source: یک‌شنبه فروردین 

source: سه‌شنبه شهریور ماه 20 1352
output: 1352-06-20
expected: 1352-06-20
bleu_score: 0.9146912192286945
----
source: 3 08 77
output: 1377-03-00
expected: 1377-08-03
bleu_score: 0.8408964152537145
----
source: جمعه شهریور ماه 8 1370
output: 1370-06-08
expected: 1370-06-08
bleu_score: 0.9146912192286945
----
source: پنج شنبه دی ماه 26 1386
output: 1366-10-26
expected: 1386-10-26
bleu_score: 0.8801117367933934
----
source: چهار شنبه خرداد ماه ۲۷ ۱۳۷۶
output: 1376-03-27
expected: 1376-03-27
bleu_score: 0.9146912192286945
----
source: چهار شنبه فروردین ماه 26 1385
output: 1385-01-26
expected: 1385-01-26
bleu_score: 0.9457416090031758
----
source: بهمن ماه 27 1382
output: 1382-11-27
expected: 1382-11-27
bleu_score: 0.8801117367933934
----
source: یک‌شنبه اسفند ماه ۸ هزار و سیصد و هشتاد و دو
output: 1382-12-08
expected: 1382-12-08
bleu_score: 0.8801117367933934
----
source: 01.08.64
output: 1364-08-10
expected: 1364-08-01
bleu_score: 0.9146912192286945
----
source: چهار شنبه فروردین ماه ۲ ۱

source: ۳۰ مهر ماه ۱۳۸۲
output: 1382-07-30
expected: 1382-07-30
bleu_score: 0.9146912192286945
----
source: هفتم مرداد هزار و سیصد و هشتاد
output: 1380-05-07
expected: 1380-05-07
bleu_score: 0.9146912192286945
----
source: دو‌شنبه آذر ماه ۶ ۱۳۵۶
output: 1366-09-06
expected: 1356-09-06
bleu_score: 0.8801117367933934
----
source: ۱۰ ۰۳ ۵۵
output: 1355-00-10
expected: 1355-03-10
bleu_score: 0.8408964152537145
----
source: بیست و شش خرداد هزار و سیصد و نود و هفت
output: 1397-03-26
expected: 1397-03-26
bleu_score: 0.9457416090031758
----
source: جمعه بهمن ماه 26 هزار و سیصد و هفتاد و سه
output: 1373-11-26
expected: 1373-11-26
bleu_score: 0.8801117367933934
----
source: چهارشنبه دی ماه 11 هزار و سیصد و هشتاد
output: 1380-10-11
expected: 1380-10-11
bleu_score: 0.8408964152537145
----
source: بیست و هشتم بهمن ماه شصت و نه
output: 1369-11-28
expected: 1369-11-28
bleu_score: 0.9146912192286945
----
source: یک شنبه خرداد ماه ۸ ۱۴۰۰
output: 1400-03-08
expected: 1400-03-08
bleu_score: 0.88011173679

...

source: اردیبهشت ۴ هزار و سیصد و شصت و شش
output: 1366-02-04
expected: 1366-02-04
bleu_score: 0.9146912192286945
----
source: پانزده اردیبهشت 1400
output: 1400-02-15
expected: 1400-02-15
bleu_score: 0.8801117367933934
----
source: 15 شهریور ماه پنجاه و پنج
output: 1355-06-15
expected: 1355-06-15
bleu_score: 0.8801117367933934
----
source: پنج شنبه تیر ماه 12 هزار و سیصد و هشتاد و هفت
output: 1387-04-12
expected: 1387-04-12
bleu_score: 0.9457416090031758
----
source: فروردین ۱ هزار و سیصد و هفتاد و سه
output: 1373-01-01
expected: 1373-01-01
bleu_score: 0.8408964152537145
----
source: آبان ماه ۲۷ ۱۳۷۲
output: 1372-08-27
expected: 1372-08-27
bleu_score: 0.9146912192286945
----
source: بیست مرداد هزار و سیصد و شصت و هشت
output: 1368-05-20
expected: 1368-05-20
bleu_score: 0.9457416090031758
----
source: ۲/۵/۶۶
output: 1366-02-25
expected: 1366-02-05
bleu_score: 0.9146912192286945
----
source: 04.06.93
output: 1393-04-04
expected: 1393-06-04
bleu_score: 0.8801117367933934
----
source: 03.01.

source: بیست و هفتم فروردین ۱۳۵۲
output: 1352-01-27
expected: 1352-01-27
bleu_score: 0.9146912192286945
----
source: چهارشنبه آذر ماه ۱ ۱۳۶۲
output: 1362-09-11
expected: 1362-09-01
bleu_score: 0.9146912192286945
----
source: هفتم آذر ماه شصت و پنج
output: 1365-07-07
expected: 1365-09-07
bleu_score: 0.9146912192286945
----
source: 27.03.92
output: 1392-07-27
expected: 1392-03-27
bleu_score: 0.9146912192286945
----
source: 20 تیر هزار و سیصد و پنجاه و چهار
output: 1354-04-20
expected: 1354-04-20
bleu_score: 0.9146912192286945
----
source: بیست و چهار خرداد ماه هزار و سیصد و هفتاد و نه
output: 1379-03-24
expected: 1379-03-24
bleu_score: 0.9457416090031758
----
source: بیست و هفتم فروردین شصت و نه
output: 1369-01-27
expected: 1369-01-27
bleu_score: 0.9457416090031758
----
source: پنجشنبه آبان ماه ۴ ۱۳۹۶
output: 1399-08-04
expected: 1396-08-04
bleu_score: 0.9146912192286945
----
source: بیست و ششم آبان ۱۳۸۱
output: 1381-08-26
expected: 1381-08-26
bleu_score: 0.9146912192286945
----
source: 

source: چهار شنبه آبان ماه ۲۴ هزار و سیصد و نود و یک
output: 1391-08-24
expected: 1391-08-24
bleu_score: 0.9457416090031758
----
source: اردیبهشت ماه ۱۶ ۱۳۶۰
output: 1360-02-16
expected: 1360-02-16
bleu_score: 0.8801117367933934
----
source: سیزده بهمن 1353
output: 1353-11-13
expected: 1353-11-13
bleu_score: 0.7952707287670506
----
source: سه شنبه فروردین ماه 9 1359
output: 1359-01-09
expected: 1359-01-09
bleu_score: 0.8801117367933934
----
source: یک‌شنبه خرداد ماه 8 هفتاد و دو
output: 1372-03-08
expected: 1372-03-08
bleu_score: 0.9146912192286945
----
source: ۲۲ مهر ماه ۱۳۶۸
output: 1366-07-22
expected: 1368-07-22
bleu_score: 0.9146912192286945
----
source: آذر ماه ۱ هزار و سیصد و نود
output: 1390-09-01
expected: 1390-09-01
bleu_score: 0.8408964152537145
----
source: 7 03 79
output: 1379-07-07
expected: 1379-03-07
bleu_score: 0.8801117367933934
----
source: خرداد ماه ۲۸ ۱۳۹۱
output: 1391-03-28
expected: 1391-03-28
bleu_score: 0.9146912192286945
----
source: 08.08.56
output: 1356-08-0

output: 1398-09-09
expected: 1398-09-13
bleu_score: 0.8801117367933934
----
source: چهار‌شنبه اسفند ماه 2 1372
output: 1372-12-02
expected: 1372-12-02
bleu_score: 0.8801117367933934
----
source: خرداد ماه 15 هزار و سیصد و نود و یک
output: 1391-03-15
expected: 1391-03-15
bleu_score: 0.8801117367933934
----
source: یکشنبه فروردین ماه ۲۸ ۱۳۸۷
output: 1387-01-28
expected: 1387-01-28
bleu_score: 0.9146912192286945
----
source: دوشنبه فروردین ماه 16 هزار و سیصد و هشتاد و شش
output: 1366-01-16
expected: 1386-01-16
bleu_score: 0.8408964152537145
----
source: یکشنبه آذر ماه 5 هفتاد و نه
output: 1379-09-05
expected: 1379-09-05
bleu_score: 0.9146912192286945
----
source: 12 بهمن هزار و سیصد و پنجاه و هشت
output: 1358-11-22
expected: 1358-11-12
bleu_score: 0.8801117367933934
----
source: ۲۵ مهر ماه هزار و سیصد و هشتاد و هشت
output: 1388-07-25
expected: 1388-07-25
bleu_score: 0.9457416090031758
----
source: 3/5/72
output: 1372-03-03
expected: 1372-03-05
bleu_score: 0.8801117367933934
----
source: ش

source: چهار‌شنبه دی ماه ۳ ۱۳۹۲
output: 1392-10-03
expected: 1392-10-03
bleu_score: 0.8801117367933934
----
source: ۰۳.۰۸.۸۵
output: 1388-08-00
expected: 1385-08-03
bleu_score: 0.8408964152537145
----
source: سه‌شنبه مهر ماه ۲۶ ۱۳۵۷
output: 1357-07-26
expected: 1357-07-26
bleu_score: 0.9457416090031758
----
source: بهمن ماه 2 هزار و سیصد و هشتاد و پنج
output: 1385-11-22
expected: 1385-11-02
bleu_score: 0.8801117367933934
----
source: سه‌شنبه اسفند ماه 8 1350
output: 1350-12-08
expected: 1350-12-08
bleu_score: 0.9146912192286945
----
source: سه‌شنبه تیر ماه 3 هزار و سیصد و نود و دو
output: 1392-04-03
expected: 1392-04-03
bleu_score: 0.9146912192286945
----
source: سهشنبه دی ماه ۳ ۱۳۶۹
output: 1366-10-03
expected: 1369-10-03
bleu_score: 0.8408964152537145
----
source: چهار دی ماه 1375
output: 1355-10-04
expected: 1375-10-04
bleu_score: 0.8801117367933934
----
source: دوشنبه دی ماه ۲۹ هزار و سیصد و نود و هشت
output: 1398-10-29
expected: 1398-10-29
bleu_score: 0.9146912192286945
----
sourc

In [43]:
print("bleu_scores:", np.average(bleu_scores))

bleu_scores: 0.8977271380336775


In [44]:
true / len(testSet)

0.7098